In [10]:
!pip install bert-score
!pip install moverscore
!pip install pycocoevalcap
!pip install pyemd

In [ ]:
import os
import gc
import nltk
import pandas as pd
import torch
from bert_score import score as bert_score

nltk.download('wordnet')

# CUDA kullanılabilirliğini kontrol et ve cihazı ayarla
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Lists to store computed metrics per sample
bert_f1_list = []

# Also, store the original texts to rebuild a DataFrame later
ref_texts = []
cand_texts = []

total_samples = 0
sum_bert_f1 = 0.0

chunksize = 1000  # Adjust chunksize based on available memory
csv_file = '/content/drive/MyDrive/updated_annotations.csv'
count = 0

for chunk in pd.read_csv(csv_file, chunksize=chunksize):
    # Extract lists of texts for batch processing
    chunk_refs = chunk['title_multi_objects'].tolist()
    chunk_cands = chunk['generated_captions'].tolist()

    # Save texts for later DataFrame construction
    ref_texts.extend(chunk_refs)
    cand_texts.extend(chunk_cands)

    # Calculate BERTScore for the entire chunk using CUDA if available
    _, _, bert_f1_chunk = bert_score(
        chunk_cands,
        chunk_refs,
        lang="en",
        model_type="bert-base-uncased",
        device=device,
        batch_size=128  # Increase batch_size if memory permits
    )

    # Convert tensor to list and update cumulative values
    bert_f1_chunk = bert_f1_chunk.tolist()
    bert_f1_list.extend(bert_f1_chunk)
    sum_bert_f1 += sum(bert_f1_chunk)
    total_samples += len(bert_f1_chunk)

    # Clean up memory for the processed chunk
    del chunk
    gc.collect()

    print("Processed chunk:", count)
    count += 1

# Create a DataFrame from the accumulated lists
result_df = pd.DataFrame({
    'title_multi_objects': ref_texts,
    'generated_captions': cand_texts,
    'BERT-F1': bert_f1_list
})

print(result_df[['title_multi_objects', 'generated_captions', 'BERT-F1']].head())

# Calculate average BERT-F1 value
avg_bert_f1 = sum_bert_f1 / total_samples
print("Dataset Average Values:")
print(f"  BERT-F1 : {avg_bert_f1:.4f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Processed chunk: 0
Processed chunk: 1
Processed chunk: 2
Processed chunk: 3
Processed chunk: 4
Processed chunk: 5
Processed chunk: 6
Processed chunk: 7
Processed chunk: 8
Processed chunk: 9
Processed chunk: 10
Processed chunk: 11
Processed chunk: 12
Processed chunk: 13
Processed chunk: 14
Processed chunk: 15
Processed chunk: 16
Processed chunk: 17
Processed chunk: 18
Processed chunk: 19
Processed chunk: 20
Processed chunk: 21
Processed chunk: 22
Processed chunk: 23
Processed chunk: 24
Processed chunk: 25
Processed chunk: 26
Processed chunk: 27
Processed chunk: 28
Processed chunk: 29
Processed chunk: 30
Processed chunk: 31
Processed chunk: 32
Processed chunk: 33
Processed chunk: 34
Processed chunk: 35
Processed chunk: 36
Processed chunk: 37
Processed chunk: 38
Processed chunk: 39
Processed chunk: 40
Processed chunk: 41
Processed chunk: 42
Processed chunk: 43
Processed chunk: 44
Processed chunk: 45
Processed chunk: 46
Processed chunk: 47
Processed chunk: 48
Processed chunk: 49
Processed 

In [1]:
import os
import gc
import nltk
import pandas as pd
import torch
import random
from transformers import CLIPProcessor, CLIPModel

nltk.download('wordnet')

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# CSV'den verileri okuyarak metinleri toplayın
ref_texts = []
cand_texts = []
chunksize = 1000
csv_file = '/content/drive/MyDrive/updated_annotations.csv'
count = 0

for chunk in pd.read_csv(csv_file, chunksize=chunksize):
    ref_texts.extend(chunk['title_multi_objects'].tolist())
    cand_texts.extend(chunk['generated_captions'].tolist())
    del chunk
    gc.collect()
    print("Processed chunk:", count)
    count += 1

# Boş, NaN veya yalnızca boşluk içeren metin çiftlerini filtreleyelim.
filtered_refs = []
filtered_cands = []
for ref, cand in zip(ref_texts, cand_texts):
    if pd.notna(ref) and pd.notna(cand) and str(ref).strip() and str(cand).strip():
        filtered_refs.append(str(ref).strip())
        filtered_cands.append(str(cand).strip())

print("Number of valid pairs:", len(filtered_refs))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Using device: cuda
Processed chunk: 0
Processed chunk: 1
Processed chunk: 2
Processed chunk: 3
Processed chunk: 4
Processed chunk: 5
Processed chunk: 6
Processed chunk: 7
Processed chunk: 8
Processed chunk: 9
Processed chunk: 10
Processed chunk: 11
Processed chunk: 12
Processed chunk: 13
Processed chunk: 14
Processed chunk: 15
Processed chunk: 16
Processed chunk: 17
Processed chunk: 18
Processed chunk: 19
Processed chunk: 20
Processed chunk: 21
Processed chunk: 22
Processed chunk: 23
Processed chunk: 24
Processed chunk: 25
Processed chunk: 26
Processed chunk: 27
Processed chunk: 28
Processed chunk: 29
Processed chunk: 30
Processed chunk: 31
Processed chunk: 32
Processed chunk: 33
Processed chunk: 34
Processed chunk: 35
Processed chunk: 36
Processed chunk: 37
Processed chunk: 38
Processed chunk: 39
Processed chunk: 40
Processed chunk: 41
Processed chunk: 42
Processed chunk: 43
Processed chunk: 44
Processed chunk: 45
Processed chunk: 46
Processed chunk: 47
Processed chunk: 48
Processed c

In [8]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
# Tokenizer'ın maksimum uzunluğunu 128 olarak ayarlıyoruz.
clip_processor.tokenizer.model_max_length = 128

# CLIP'in text encoder'ı orijinalde 77 token için konumsal embedding kullanır.
# 128 token desteklemesi için positional embedding matrisini genişletiyoruz.
new_max_length = 128
old_max_length = clip_model.config.text_config.max_position_embeddings  # Genellikle 77
hidden_size = clip_model.text_model.embeddings.position_embedding.weight.shape[1]
old_pos_embed = clip_model.text_model.embeddings.position_embedding.weight.data
if new_max_length > old_max_length:
    print(f"Extending positional embeddings from {old_max_length} to {new_max_length}")
    new_pos_embed = torch.zeros(new_max_length, hidden_size, device=old_pos_embed.device)
    new_pos_embed[:old_max_length, :] = old_pos_embed
    new_pos_embed[old_max_length:, :] = old_pos_embed[-1, :].unsqueeze(0).expand(new_max_length - old_max_length, hidden_size)
    new_embedding = torch.nn.Embedding(new_max_length, hidden_size).to(device)
    new_embedding.weight.data = new_pos_embed
    clip_model.text_model.embeddings.position_embedding = new_embedding
    clip_model.config.text_config.max_position_embeddings = new_max_length

# Batch işlemesiyle CLIPScore hesaplaması:
batch_size = 32
clip_scores = []
num_batches = (len(filtered_refs) + batch_size - 1) // batch_size

for i in range(0, len(filtered_refs), batch_size):
    batch_refs = filtered_refs[i:i+batch_size]
    batch_cands = filtered_cands[i:i+batch_size]
    # Her çift için referans ve aday metinleri ardışık olarak listeye ekliyoruz.
    texts = []
    for ref, cand in zip(batch_refs, batch_cands):
        texts.append(ref)
        texts.append(cand)
    inputs = clip_processor(text=texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    # Explicit olarak position_ids üretiyoruz:
    input_ids = inputs["input_ids"]
    position_ids = torch.arange(input_ids.shape[1], dtype=torch.long, device=device).unsqueeze(0).expand_as(input_ids)
    inputs["position_ids"] = position_ids
    with torch.no_grad():
        text_features = clip_model.get_text_features(**inputs)
    # Her iki ardışık metin çifti için cosine similarity hesaplayalım.
    for j in range(0, text_features.shape[0], 2):
        score = torch.nn.functional.cosine_similarity(text_features[j:j+1], text_features[j+1:j+2]).item()
        clip_scores.append(score)
    print(f"Processed batch {i//batch_size + 1}/{num_batches}")

avg_clip_score = sum(clip_scores) / len(clip_scores)
print("Average CLIPScore: {:.4f}".format(avg_clip_score))

Streaming output truncated to the last 5000 lines.
Processed batch 16127/21125
Processed batch 16128/21125
Processed batch 16129/21125
Processed batch 16130/21125
Processed batch 16131/21125
Processed batch 16132/21125
Processed batch 16133/21125
Processed batch 16134/21125
Processed batch 16135/21125
Processed batch 16136/21125
Processed batch 16137/21125
Processed batch 16138/21125
Processed batch 16139/21125
Processed batch 16140/21125
Processed batch 16141/21125
Processed batch 16142/21125
Processed batch 16143/21125
Processed batch 16144/21125
Processed batch 16145/21125
Processed batch 16146/21125
Processed batch 16147/21125
Processed batch 16148/21125
Processed batch 16149/21125
Processed batch 16150/21125
Processed batch 16151/21125
Processed batch 16152/21125
Processed batch 16153/21125
Processed batch 16154/21125
Processed batch 16155/21125
Processed batch 16156/21125
Processed batch 16157/21125
Processed batch 16158/21125
Processed batch 16159/21125
Processed batch 16160/211